In [1]:
import numpy as np

@article{2006_avellaneda,
    author = {Marco Avellaneda and Sasha Stoikov},
    title = {High-frequency trading in a limit order book},
    doi = {10.1080/14697680701381228},
    journal = {Quantitative Finance},
    number = {3},
    pages = {217--224},
    publisher = {Routledge},
    url = {https://doi.org/10.1080/14697680701381228},
    volume = {8},
    year = {2008},
    bdsk-url-1 = {https://doi.org/10.1080/14697680701381228}}

#### ASSUMPTIONS:
1. asssume money market pays no interest
2. mid-price given by $dS_u = \sigma d W_u$
   1. initial value, $S_t = s, W_t$ is standard 1D Brownian motion,
   2. $\sigma$ is constant
3. agent can not affect drift or autocorrelation structure of stock
4. 

### globa variables:
 - gamma = discount price?
 - sigma = brownian motion variance
 - terminal time T at which stock dies or sumn


'frozen inventory' strategy
inactive trader
no limit orders and only holds inventory of q stocks until T
value function given by $v(x,s,q,t)$, where
 - x = initial wealth, q = nstocks, t = time
 - s = initial stock value, = midprice

reservation bid price is price that makes agent indifferent to buy a stock
$$v(x-r^b(s,q,t), s, q+1, t) >= v(x,s,q,t)$$
reservation ask price is price that makes agent indifferent to sell a stock
$$v(x+r^a(s,q,t), s, q-1, t) >= v(x,s,q,t)$$
where $r^b$, $r^a$ is bid, ask price

In [3]:
gamma = sigma = 1; T = 1000
# v(x,s,q,t), 
# x = initial wealth, s = initial stock value, q = nstocks, t = time
def frozen_value(initial_wealth, stock_val, nstocks, time):
    first = -np.exp(-gamma*(initial_wealth+nstocks*stock_val))
    second = np.exp((gamma*nstocks*sigma)**2 * (T - time) / 2)
    return first * second

def res_ask_price(s,q,t):
    return s + (1-2*q) * gamma * sigma**2 * (T-t)
def res_bid_price(s,q,t):
    return s - (1+2*q) * gamma * sigma**2 * (T-t)
# avg between bid and ask
def res_price(s, q, t):  # reservation / indifference price
    return s - q * gamma * sigma**2 * (T-t)

### 2.4 :  adding limit orders

quotes bid price $p^b$, ask price $p^a$
focus on distances $\delta^b = s - p^b$ and $\delta^a = p^a - s$

imagine market order of $Q$ stocks arrives, the $Q$ limit orders with lowest 
ask prices are sold. if $p^Q$ is price of highest limit order, 
$\Delta p = p^Q - s$  is the temporary market impact of the trade

ASSUME 
- market buy orders "lift" agent's limit asks at Poisson rate
$\lambda^a(\delta^a)$, monotonically decreasing function
- markey sells will "hit" the buy limit orders at rate $\lambda^b(\delta^a)$
these rates are also called the Poisson "intensities"

$X_t$ = wealth of agent. $N_t^a$ is # stocks sold, $N_t^b$ is # stocks bought. Follows stochastic function
$$dX_t = p^a dN_t^a - p^b dN_t^b$$

stocks held at time $t$ is $q_t$
objective is
$$u(s, x, q, t) = \max_{\delta^a,\delta^b} 
\mathbb{E}_t \left[-\exp\left(-\gamma(X_T + q_T S_T)\right)\right]$$

### 2.5: Trading Intensity

ASSUME constant frequency $\Lambda$ of market buy or sell orders. 

The density of market order size is given by
$$f^Q(x)\propto x^{-1-\alpha}$$


for large $x$. $\alpha=1.53$ for US stocks, $\alpha = 1.4$ for NASDAQ stocks, and $\alpha = 1.5$ for Paris Bourse.

Market impact is given by either $\Delta p \propto Q^\beta$ for $\beta=0.5,0.76$ but is better fit to $\Delta p\propto \ln(Q)$.

Authors find that 
For market impact given $\Delta p\propto \ln(Q)$, $$\lambda(\delta)=A e^{-k\delta}$$

For a power-law market impact $\Delta p \propto Q^\beta$, $$\lambda(\delta)=B\delta^{-\alpha/\beta}$$

Or we can integrate the density of the limit order book?

for  $A=\Lambda/\alpha$, $k=\alpha K$.

Agent objective function is given by $u(s,x,q,t)$ is a solution to Hamilton-Jacobi-Bellman equation. 

$\delta^a$ and $\delta^b$ is 

The utility function is continuous in $s,x,t$ but discrete in $q$:
$$u(s,x,q,t)=-\exp\left(-\gamma\left[x+\theta(s,q,t)\right]\right)$$

where we have introduced the ansatz $\theta(s,q,t)$ as an "inventory variable" that is basically $q(t)*s(t)$, giving the amount of wealth that is "in" the inventory of the agent.

To optimize $u$, we must solve the equation

$$\frac{\partial\theta}{\partial t} +\frac12 \sigma^2 \frac{\partial^2\theta}{\partial s^2} - \frac12 \sigma^2 \gamma \left(\frac{\partial\theta}{\partial s}\right)^2 + \max_{\delta^b}\left[\frac{\lambda^b(\delta^b)}{\gamma}\left(1-e^{\gamma(s-\delta^b-r^b)}\right)\right]+\max_{\delta^a}\left[\frac{\lambda^a(\delta^a)}{\gamma}\left(1-e^{\gamma(\delta^a+r^a-s)}\right)\right]=0 \tag{1}$$

with boundary condition $\theta(s,q,T) = qs$


We can derive the reservation bid and ask prices implicitly as
$$r^b(s,q,t)=\theta(s,q+1,t)-\theta(s,q,t)=s-\delta^b+\frac1\gamma\ln\left(1-\gamma\frac{\lambda^b(\delta^b)}{(\partial\lambda^b/\partial\delta)(\delta^b)}\right)$$
 and 
$$r^a(s,q,t)=\theta(s,q,t)-\theta(s,q-1,t)=\delta^a-s-\frac1\gamma\ln\left(1-\gamma\frac{\lambda^a(\delta^a)}{(\partial\lambda^a/\partial\delta)(\delta^a)}\right)$$

Substituting these expressions into Eq. (1), with exponential arrival rates $\lambda(\delta)=A e^{-k\delta}$, we get 
$$\frac{\partial\theta}{\partial t} +\frac12 \sigma^2 \frac{\partial^2\theta}{\partial s^2} - \frac12 \sigma^2 \gamma \left(\frac{\partial\theta}{\partial s}\right)^2 + \frac{Ae^{-k^a\delta^a}}{k^a+\gamma} + \frac{Be^{-k^b\delta^b}}{k^b+\gamma}= 0\tag{2}$$

So we have to first 
1. solve the PDE (1) to find $r^b,\ r^a$
2. Calculate optimal distances $\delta^b,\ \delta^a$

However, this is very computationally difficult, so instead can assume that the number of stocks the agent has is always small, such that we can taylor expand $\theta$ around $q=0$:

$$\theta(q,s,t) = \theta^{(0)}(s,t)+q\theta^{(1)}(s,t)+\frac12 q^2 \theta^{(2)}(s,t)+\frac16 q^3 \theta^{(3)}(s,t)+\cdots$$



Using a bunch of equations and optimizations, we can linearly expand
bid-ask spread is given by 
$$\delta^a+\delta^b = \gamma \sigma^2 (T-t)+\frac2\gamma \ln\left(1+\frac\gamma{k}\right)$$

